# Evaluate 2D MENT model

In [ ]:
import os
import pickle
import sys

import matplotlib.pyplot as plt
import numpy as np
import proplot as pplt
import psdist as ps
import psdist.visualization as psv
import torch
from ipywidgets import interact
from ipywidgets import widgets

import mentflow as mf
from mentflow.utils import grab
from mentflow.utils import unravel

from experiments.load import load_ment_run
from setup import make_dist

## Settings

In [ ]:
device = "cpu"
precision = torch.float32

def send(x):
    return x.type(precision).to(device)

## Load data

In [ ]:
script_name = "train_ment"
timestamp = 240123092736
data_dir = f"./outputs/{script_name}/{timestamp}/"

In [ ]:
run = load_ment_run(data_dir, device="cpu")

cfg = run["config"]
history = run["history"]
model = run["model"]
checkpoints = run["checkpoints"]

In [ ]:
dist = make_dist(cfg)

## Evaluation

### Density

In [ ]:
@interact(
    index=widgets.IntSlider(
        min=0, 
        max=(len(checkpoints) - 1), 
        value=(len(checkpoints) - 1),
    ),
    n=widgets.FloatLogSlider(min=2, max=6, value=1.00e+05),
    xmax=widgets.FloatSlider(min=0.0, max=6.0, value=3.0),
    bins=widgets.IntSlider(min=4, max=150, value=125),
)
def update(index, n, xmax, bins):
    model.load(checkpoints[index]["path"], device="cpu")

    with torch.no_grad():
        x_pred = grab(model.sample(int(n)))
        x_true = grab(dist.sample(int(n)))
        
        fig, axs = pplt.subplots(ncols=2, xspineloc="neither", yspineloc="neither", space=0)
        kws = dict()
        limits = 2 * [(-xmax, xmax)]
        axs[0].hist2d(x_true[:, 0], x_true[:, 1], bins=bins, range=limits, **kws)
        axs[1].hist2d(x_pred[:, 0], x_pred[:, 1], bins=bins, range=limits, **kws)
        pplt.show()

### Projections

In [ ]:
@interact(
    index=widgets.IntSlider(
        min=0, 
        max=(len(checkpoints) - 1), 
        value=(len(checkpoints) - 1),
    ),
    n=widgets.FloatLogSlider(min=2, max=6, value=50000),
    log_ymin=widgets.FloatSlider(min=-5.0, max=-1.0, value=-3.0),
)
def update(index, n, log_ymin, kde=False, log=False):
    model.load(checkpoints[index]["path"], device="cpu")

    kind = "line"
    lw = 1.25
    colors = ["black", "red"]
    
    with torch.no_grad():
        x_pred = model.sample(int(n))

        for diagnostic in unravel(model.diagnostics):
            diagnostic.kde = kde
            
        predictions = mf.sim.forward(x_pred, model.transforms, model.diagnostics)

        y_pred = [grab(prediction) for prediction in unravel(predictions)]
        y_meas = [grab(measurement) for measurement in unravel(model.measurements)]
        edges = [grab(diagnostic.bin_edges) for diagnostic in unravel(model.diagnostics)]

        fig, axs = mf.train.plot.plot_proj_1d(
            y_pred,
            y_meas,
            edges,
            maxcols=7,
            kind=kind,
            height=1.25,
            lw=lw,
            colors=colors,
        )
        
        x_true = dist.sample(int(n))
        predictions = mf.sim.forward(x_true, model.transforms, model.diagnostics)
        y_pred = [grab(prediction) for prediction in unravel(predictions)]
        for i in range(len(y_pred)):
            mf.train.plot.plot_profile(
                y_pred[i] / y_meas[i].max(), 
                edges[i], 
                ax=axs[i],
                kind=kind,
                lw=lw,
                zorder=0, 
                color=colors[0],
                alpha=0.2,
            )
        
        axs.format(ymax=1.25, ymin=(10.0 ** log_ymin))
        if log:
            axs.format(yscale="log")
        plt.show()